In [71]:
QUERY_TRANSLATE = [
    {
        "table_name": "colors",
        "table_type": "array",
        "query_command": ["color", "c"]
    },

    {
        "table_name": "color_identity",
        "table_type": "array",
        "query_command": ["id", "identity"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': 'type_line', 'type': 'string'}],
        "query_command": ["t", "type"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': '"set"', 'type': 'string'}, {'name':'set_name', 'type': 'string'}],
        "query_command": ["e", "edition", "s", "set"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': 'rarity', 'type': 'string'}],
        "query_command": ["r", "rarity"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': 'cmc', 'type': 'float'}],
        "query_command": ["cmc", "mv", "manavalue"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': 'oracle_text', 'type': 'string'}],
        "query_command": ["o", "oracle"]
    },

    {
        "table_name": "prices",
        "table_type": "object",
        "columns": [{'name': 'usd', 'type': 'float'}, {'name':'eur', 'type': 'float'}, {'name':'tix', 'type': 'float'}],
        "query_command": ["eur", "usd", "tix"]
    },

    {
        "table_name": "games",
        "table_type": "array",
        "query_command": ["game"]
    },

    {
        "table_name": "keywords",
        "table_type": "array",
        "query_command": ["keyword"]
    },

    {
        "table_name": "main_table",
        "table_type": "main",
        "columns": [{'name': 'released_at', 'type': 'datetime'}],
        "query_command": ["date", "year"]
    },

]

In [7]:
def decide_operator(query_string):
    operator = ""
    if ">=" in query_string:
        operator = ">="
    elif "<=" in query_string:
        operator = "<="
    elif ">" in query_string:
        operator = ">"
    elif "<" in query_string:
        operator = "<"
    elif ":" in query_string:
        operator = ":"

    return operator

In [35]:
def handle_query_when_number(table, operator, column, value, value_type):
    if value_type == "float":
        correct_value = float(value)
    elif value_type == "int":
        correct_value = int(value)
    
    correct_operator = operator if operator != ":" else "="
    query = f"{table}.{column} {operator} {str(correct_value)} AND "
    return query


In [96]:
def handle_query_when_string(table, column, value, command):
    value = value.replace('"',"")
    query = f"{table}.{column} {'LIKE' if command.find('-') == -1 else 'NOT LIKE'} '%{value}%' AND "
    return query

In [83]:
def handle_query_when_date(table, operator, column, value, command):
    value = value.replace('"',"")
    query = ""
    correct_operator = operator if operator != ":" else "="
    if command == "year":
        query = f"CAST(STRFTIME('%Y',{table}.{column}) AS INT) {correct_operator} {str(int(value))} AND "
    else:
        query = f"{table}.{column} {correct_operator} '{str(value)}' AND "
    return query

In [110]:
def handle_query_when_array(table, value, command):
    query = f"{table}.array_value {'LIKE' if command.find('-') == -1 else 'NOT LIKE'} '%{value}%' AND "
    return query

In [44]:
def handle_query_when_prices(operator, column, value):
    value = float(value)
    correct_operator = operator if operator != ":" else "="
    query = f"cast(prices.{column} as float) {correct_operator} {str(value)} AND "
    return query

In [54]:
def handle_query_when_format(operator, column, value):
    correct_operator = operator if operator != ":" else "="
    query = f"cast(prices.{column} as float) {correct_operator} {str(value)} AND "
    return query

In [4]:
def construct_query_join(table_name):
    join_query = f"""
    INNER JOIN {table_name}_table as {table_name}
    ON main.id = {table_name}.card_id
    """
    return join_query

In [114]:
def construct_query_when(query_string):
    where_query = "WHERE "
    query_elements = query_string.split("  ")
    for element in query_elements:
        operator = decide_operator(element)
        element_array = element.split(operator)
        argument = element_array[0]
        value = element_array[1]
     
        for translation in QUERY_TRANSLATE:
            translation_commands = translation["query_command"]
            translation_commands_negative = [f"-{element}" for element in translation_commands]
            if argument in translation_commands or argument in translation_commands_negative:
                table_name = translation["table_name"]
                if translation["table_type"] == "main":
                    if len(translation["columns"]) == 1:
                        column = translation["columns"][0]
                        column_name = column["name"]
                        column_type = column["type"]
                        
                        if column_type in ['float', 'int']:
                            query = handle_query_when_number(table_name, operator, column_name, value, column_type)
                            where_query += query
                        elif column_type in ['datetime']:
                            query = handle_query_when_date(table_name, operator, column_name, value, argument)
                            where_query += query
                        else:
                            print(argument)
                            query = handle_query_when_string(table_name, column_name, value, argument)
                            where_query += query
                            
                elif translation["table_type"] == "array":
                    values = value.split(",")
                    query = ""
                    for value in values:
                        query += handle_query_when_array(table_name, value, argument)
                    where_query += query

                elif translation["table_type"] == "object":
                    match table_name:
                        case "prices":
                            query = handle_query_when_prices(operator, argument, value)
                            where_query += query

    where_query = where_query[:-4]            
    return where_query

In [117]:
query_string = 'rarity:mythic'
construct_query_when(query_string)

rarity


"WHERE main_table.rarity LIKE '%mythic%' "

In [47]:
query_string = 'color:r,g -color:w'
def construct_query(query_string=None):
    query = 'SELECT id FROM main_table'
    query_commands = list(set([element.split(":")[0] for element in query_string.split(" ")]))

    tables_to_include = []
    for element in query_commands:
        for record in QUERY_TRANSLATE:
            if element in record['query_command']:
                tables_to_include.append(record['table_name'])

    for table in tables_to_include:
        join_query = construct_query_join(table)
        query += join_query
    
    return query

In [24]:

construct_query(query_string)

'SELECT id FROM main_table\n    INNER JOIN prices_table as prices\n    ON main.id = prices.card_id\n    \n    INNER JOIN colors_table as colors\n    ON main.id = colors.card_id\n    '